Script to
1. Propagate Relationships, add missing Authority Terms to Relationship entries, and return csv for Standardized Directory
2. Return csv according to wikidata person schema for upload to OpenRefine

Plan:
1. import curated relationships and persons lists
2. dedup standard names from df_persons;
3. add standard names to df_relationships based on a) matching; b) missing values (using first, last names, etc)
4. propagate df_relationships
  4a. approach: row-wise: for every row, look at the kind of relationship; produce an inverted copy of it; seek third-party relationships and produce those (see 4b); deduplicate relationships
  4b. look at current row relationship (p1 to p2); loop through p2 relationships (with p3); assert p1 -> p3 relationships; loop twice, then deduplicate rows?
  4c. example: if current row has p1 as husband to p2, then script should find all relationships for p2; if p2 is mother to p3 and sister to p4, assert p1 is father to p3 (is that risky?) and p1 is brother-in-law to p4.
  4d. need a list of all
5. build df that matches person schema for wikidata, combining df_persons and df_relationships and output as .csv

In [1]:
# import requirements
import pandas as pd
import numpy as np
import re
from datetime import datetime


In [2]:
# Load the files from the current folder
df = pd.read_excel('./data/relationships_2_28_25.xlsx')
df_relationships = df.replace('', np.nan, regex=True)
df_2 = pd.read_excel('./data/standard_persons_4_21_25.xlsx')
df_persons = df_2.replace('', np.nan, regex=True).sample(n=50, random_state=42)
np.random.seed(1)

In [ ]:
df_relationships.info()

In [ ]:
df_relationships.drop(['Column17', 'Read Me', 'Column15', 'Column16'], axis=1, inplace=True)

In [ ]:
df_persons.info()

In [3]:
df_persons.drop(['Mention only?', 'Researcher/Date'], axis=1, inplace=True)

In [ ]:
df_persons[df_persons['Authority Terms']=='Audland, Anne'][['Birth Date', 'Death Date', 'Marriage Date']].head()

0. data cleaning tasks;
1. dedup standard names from df_persons;
2. add standard names to df_relationships based on a) matching; b) missing values (using first, last names, etc)
3. propagate df_relationships
4. build df that matches person schema for wikidata, combining df_persons and df_relationships and output as .csv



#Data cleaning

In [4]:
from pickle import FALSE
#delete empty rows
df_relationships.dropna(how='all', inplace=True)
df_persons.dropna(how='all', inplace=True)

#delete any row in def_persons that doesn't have an Authority Name:
df_persons = df_persons[df_persons['Authority Terms'].notna()]

#delete any authority terms that have a bracket
df_persons = df_persons[~df_persons['Authority Terms'].str.contains(r'\[|\]|\(|\)')]
#df_relationships.info()
#df_persons.info()

#rename column
df_persons.rename(columns={"Authority Terms": "name"}, inplace=True)
#CHANGE THIS!!! WRONG -- NEED PERSOn 1 Standard... and Person 2 standard to be converted
df_relationships.rename(columns={"Person 1 Standard name from List": "p1_std_name", 'Person 2 Standard name from List': 'p2_std_name'}, inplace=True)

In [ ]:
df_relationships.keys()

In [5]:
# Function to process date columns for both handlebars and brackets
def process_date_columns(row):
    # Define the columns to process and their corresponding target columns for handlebars
    handlebars_columns = {
        'Birth Date': 'DoB Source URL',
        'Death Date': 'DoD Source URL',
        'Marriage Date': 'P26+P2562 Source URL'
    }

    # Define the columns to process and their corresponding target columns for brackets
    brackets_columns = {
        'Birth Date': 'DoB Source',
        'Death Date': 'DoD (P570) Source',
        'Marriage Date': 'P26+P2562 Source'
    }

    # Process handlebars ({{ }})
    for col, target_col in handlebars_columns.items():
        if pd.notna(row[col]) and isinstance(row[col], str):  # Check if the column value is not NaN and is a string
            # Extract text between double handlebars
            matches = re.findall(r'\{\{(.*?)\}\}', row[col])
            if matches:
                # Remove text between double handlebars and the handlebars themselves
                row[col] = re.sub(r'\{\{.*?\}\}', '', row[col]).strip()

                # Prepare the source text
                source_text = '; '.join(matches)

                # Append to the target column with a semi-colon separator if it already has data
                if pd.notna(row[target_col]) and row[target_col].strip():
                    row[target_col] += '; ' + source_text
                else:
                    row[target_col] = source_text

    # Process brackets ([[ ]])
    for col, target_col in brackets_columns.items():
        if pd.notna(row[col]) and isinstance(row[col], str):  # Check if the column value is not NaN and is a string
            # Extract text between double brackets
            matches = re.findall(r'\[\[(.*?)\]\]', row[col])
            if matches:
                # Remove text between double brackets and the brackets themselves
                row[col] = re.sub(r'\[\[.*?\]\]', '', row[col]).strip()

                # Prepare the source text
                source_text = '; '.join(matches)

                # Append to the target column with a semi-colon separator if it already has data
                if pd.notna(row[target_col]) and row[target_col].strip():
                    row[target_col] += '; ' + source_text
                else:
                    row[target_col] = source_text
    return row

# Function to strip leading and trailing whitespace from specific date and source columns
def strip_whitespace_from_specific_columns(df):
    # Define the specific columns to clean
    columns_to_clean = [
        'Birth Date', 'Death Date', 'Marriage Date',
        'DoB Source', 'DoD (P570) Source', 'P26+P2562 Source',
        'DoB Source URL', 'DoD Source URL', 'P26+P2562 Source URL'
    ]

    # Strip leading and trailing whitespace from these columns
    for col in columns_to_clean:
        if col in df.columns:  # Ensure the column exists in the DataFrame
            df[col] = df[col].astype(str).str.strip()  # Convert to string and strip whitespace
    return df

# Add new columns to the DataFrame for both handlebars and bracket-based sources
df_persons['DoB Source'] = ''
df_persons['DoD (P570) Source'] = ''
df_persons['P26+P2562 Source'] = ''
df_persons['DoB Source URL'] = ''
df_persons['DoD Source URL'] = ''
df_persons['P26+P2562 Source URL'] = ''

# Apply the functions to the DataFrame
df_persons = df_persons.apply(process_date_columns, axis=1)
df_persons = strip_whitespace_from_specific_columns(df_persons)

In [ ]:
df_persons[df_persons['name']=='Audland, Anne'][['Birth Date', 'Death Date', 'Marriage Date']].head()

In [6]:
from datetime import datetime
import pandas as pd

# Identify all date columns in df_persons
date_columns = ['Birth Date', 'Death Date', 'Marriage Date']

# Function to handle partial dates
def fix_partial_date(date_str):
    if pd.isna(date_str) or date_str.strip() == '':
        return None  # Handle missing or empty values
    date_str = date_str.strip()
    try:
        if date_str.endswith('-00-00'):  # Year only (e.g., '1627-00-00')
            year = int(date_str[:4])
            return datetime(year, 1, 1)  # Represent the year as January 1st
        elif date_str.endswith('-00'):  # Year and month only (e.g., '1627-01-00')
            year, month = map(int, date_str.split('-')[:2])
            return datetime(year, month, 1)  # Represent the month as the 1st day
        else:  # Full date
            return pd.to_datetime(date_str, errors='coerce')
    except Exception:
        return None  # Return None if conversion fails

# Apply the function to each date column
for col in date_columns:
    df_persons[col] = df_persons[col].astype(str).apply(
        lambda x: fix_partial_date(x) if pd.notna(fix_partial_date(x)) else x
    )

In [7]:
# Function to process place columns for both handlebars and brackets
def process_place_columns(row):
    # Define the columns to process and their corresponding target columns for handlebars
    handlebars_columns = {
        'Place of Birth (P19)': 'PoB Source URL',
        'Place of Death': 'Place of Death Source URL',
        'Place of Residence': 'Place of Residence Source URL'
    }

    # Define the columns to process and their corresponding target columns for brackets
    brackets_columns = {
        'Place of Birth (P19)': 'PoB Source',
        'Place of Death': 'Place of Death Source',
        'Place of Residence': 'Place of Residence Source'
    }

    # Process handlebars ({{ }})
    for col, target_col in handlebars_columns.items():
        if pd.notna(row[col]) and isinstance(row[col], str):  # Check if the column value is not NaN and is a string
            # Extract text between double handlebars
            matches = re.findall(r'\{\{(.*?)\}\}', row[col])
            if matches:
                # Remove text between double handlebars and the handlebars themselves
                row[col] = re.sub(r'\{\{.*?\}\}', '', row[col]).strip()

                # Prepare the source text
                source_text = '; '.join(matches)

                # Append to the target column with a semi-colon separator if it already has data
                if pd.notna(row[target_col]) and row[target_col].strip():
                    row[target_col] += '; ' + source_text
                else:
                    row[target_col] = source_text

    # Process brackets ([[ ]])
    for col, target_col in brackets_columns.items():
        if pd.notna(row[col]) and isinstance(row[col], str):  # Check if the column value is not NaN and is a string
            # Extract text between double brackets
            matches = re.findall(r'\[\[(.*?)\]\]', row[col])
            if matches:
                # Remove text between double brackets and the brackets themselves
                row[col] = re.sub(r'\[\[.*?\]\]', '', row[col]).strip()

                # Prepare the source text
                source_text = '; '.join(matches)

                # Append to the target column with a semi-colon separator if it already has data
                if pd.notna(row[target_col]) and row[target_col].strip():
                    row[target_col] += '; ' + source_text
                else:
                    row[target_col] = source_text
    return row

# Function to process the Occupation column for both handlebars and brackets
def process_occupation_column(row):
    # Define the target column for handlebars
    handlebars_column = 'Occupation Source URL'

    # Process handlebars ({{ }})
    if pd.notna(row['Occupation']) and isinstance(row['Occupation'], str):  # Check if the column value is not NaN and is a string
        # Extract text between double handlebars
        matches = re.findall(r'\{\{(.*?)\}\}', row['Occupation'])
        if matches:
            # Remove text between double handlebars and the handlebars themselves
            row['Occupation'] = re.sub(r'\{\{.*?\}\}', '', row['Occupation']).strip()

            # Prepare the source text
            source_text = '; '.join(matches)

            # Append to the target column with a semi-colon separator if it already has data
            if pd.notna(row[handlebars_column]) and row[handlebars_column].strip():
                row[handlebars_column] += '; ' + source_text
            else:
                row[handlebars_column] = source_text

    # Process brackets ([[ ]])
    if pd.notna(row['Occupation']) and isinstance(row['Occupation'], str):  # Check if the column value is not NaN and is a string
        # Extract text between double brackets
        matches = re.findall(r'\[\[(.*?)\]\]', row['Occupation'])
        if matches:
            # Remove text between double brackets and the brackets themselves
            row['Occupation'] = re.sub(r'\[\[.*?\]\]', '', row['Occupation']).strip()

            # Prepare the source text
            source_text = '; '.join(matches)

            # Append to the existing 'Occupation Source' column with a semi-colon separator
            if pd.notna(row['Occupation Source']) and row['Occupation Source'].strip():
                row['Occupation Source'] += '; ' + source_text
            else:
                row['Occupation Source'] = source_text
    return row

# Rename the columns in the DataFrame
df_persons.rename(columns={
    'Birth Place': 'Place of Birth (P19)',
    'Death Place': 'Place of Death',
    'Place of Residence': 'Place of Residence'
}, inplace=True)

# Add new columns to the DataFrame for both handlebars and bracket-based sources
df_persons['PoB Source'] = ''
df_persons['Place of Death Source'] = ''
df_persons['Place of Residence Source'] = ''
df_persons['PoB Source URL'] = ''
df_persons['Place of Death Source URL'] = ''
df_persons['Place of Residence Source URL'] = ''
df_persons['Occupation Source URL'] = ''  # Only add this column since 'Occupation Source' already exists

# Apply the functions to the DataFrame
df_persons = df_persons.apply(process_place_columns, axis=1)
df_persons = df_persons.apply(process_occupation_column, axis=1)

# Clean up whitespace in all relevant columns
columns_to_clean = [
    'Place of Birth (P19)', 'Place of Death', 'Place of Residence', 'Occupation',
    'PoB Source', 'Place of Death Source', 'Place of Residence Source', 'Occupation Source',
    'PoB Source URL', 'Place of Death Source URL', 'Place of Residence Source URL', 'Occupation Source URL'
]
for col in columns_to_clean:
    if col in df_persons.columns:  # Ensure the column exists in the DataFrame
        df_persons[col] = df_persons[col].str.strip()

In [16]:
df_persons.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 49 to 1298
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   name                           49 non-null     object 
 1   AltLastName                    24 non-null     object 
 2   AltMiddleName                  2 non-null      object 
 3   AltFirstName                   14 non-null     object 
 4   Maiden Name                    0 non-null      float64
 5   Title                          3 non-null      object 
 6   AssociatedPlaces               5 non-null      object 
 7   Place of Birth (P19)           9 non-null      object 
 8   Place of Death                 9 non-null      object 
 9   Place of Residence             4 non-null      object 
 10  Source for Places              5 non-null      object 
 11  Birth Date                     49 non-null     object 
 12  Death Date                     49 non-null     object 

In [17]:
print(list(zip(df_persons['name'], df_persons['Place of Birth (P19)'], df_persons['PoB Source'])))

[('Anton, Paul', 'Hirschfelde, Germany', ''), ('Schröder, Johann Wilhelm', nan, ''), ('Garside, Abraham', nan, ''), ('Seddon, Thomas', nan, ''), ('Nicholson, Timothy', nan, ''), ('Killam, John', nan, ''), ('Canstein, Carl Hildebrand Von', 'Lindenberg, Germany', ''), ('Orrell, Peter', nan, ''), ('Howgill, Francis', 'Todthorne, Westmorland, England', ''), ('Holbrooke, Joshua', nan, ''), ('Cooper, George', nan, ''), ('Endon, Joseph', nan, ''), ('Griffiths, Francis', nan, ''), ('Hebson, Richard', nan, ''), ('Feddes, Huibert', nan, ''), ('Outerlo, Wouter', nan, ''), ('Herr, Hans', nan, ''), ('Marschall, Wolf Adolph, Erb-Marschall In Thüringen', nan, ''), ('Andriessen, Pieter', nan, ''), ('Hall, Jacob', nan, ''), ('Kintigh, Martin', nan, ''), ('Thornby, Roger', nan, ''), ('Pagiter, William', nan, ''), ('Callenberg, Johann Heinrich', 'Molschleben, Germany', ''), ('Gründler, Johann Ernst', 'Weißensee, Thuringia, Germany', ''), ('Fretwell, Ralph', nan, ''), ('Newton, Samuel', nan, ''), ('Baker,

In [8]:
# RETRIEVES TITLES NEEDED
# This code retrieves unique titles from specific columns in the DataFrame and exports them to a CSV file.
# Import necessary libraries
import pandas as pd

# Columns to check for titles
cols_to_check = [
    'PoB Source',
    'Place of Death Source',
    'Place of Residence Source',
    'Occupation Source',
    'DoB Source',
    'DoD (P570) Source',
    'P26+P2562 Source'
]

# Initialize an empty set to collect unique titles
titles_set = set()

# Collect titles from each column
for col in cols_to_check:
    if col in df_persons.columns:
        # Drop NaN values and split entries with semicolons
        titles = df_persons[col].dropna().str.split(';').explode().str.strip()
        titles_set.update(titles)

# Convert the set to a sorted list
titles_list = sorted(titles_set)

# Create a DataFrame from the list
titles_df = pd.DataFrame(titles_list, columns=['Title'])

# Export the titles to a CSV file
titles_df.to_csv('titles_needed.csv', index=False, encoding='utf-8-sig')

# Display the first few rows of the titles DataFrame
titles_df.head()

,Title
0,
1,AlleFriezen
2,Boyd's Inhabitants Of London & Family Units 12...
3,Early Quaker Letters-Nuttall
4,"England & Wales, Quaker Birth, Marriage, and D..."


Bad pipe message: %s [b'"Chromium";v="136", "Brave";v="136", "Not.A/Brand";v="99"']
Bad pipe message: %s [b'sec-ch-ua-mo']
Bad pipe message: %s [b'le: ?0\r\nsec-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Win', b'ws NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36\r\nAcce', b': text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8\r\nSec-GPC: 1\r']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Chromium";v="136", "Brave";v="136", "Not.A/Brand";v="99"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-u']
Bad pipe message: %s [b'platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win', b'; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/']


In [ ]:
#DON'T RUN THIS FOR NOW
import pandas as pd
from lccn import get_lccn_for_title, confirm_lccn_matches
import time

def apply_and_confirm_lccn(df, col, get_lccn_func, confirm_func, delay=1.5, sim_threshold=95, max_retries=5, verbose=True):
    """
    Applies the LCCN retrieval function to a column, confirms the matches, and adds the results to the DataFrame.
    """
    # Step 1: Apply get_lccn_for_title to retrieve LCCNs
    lccn_col = f"{col} LCCN"
    df[lccn_col] = df[col].apply(lambda x: get_lccn_func(x) if pd.notna(x) else [])

    # Step 2: Confirm the LCCNs using confirm_lccn_matches
    df, confirmed_df = confirm_func(
        df=df,
        lccn_col=lccn_col,
        title_col=col,
        delay=delay,
        sim_threshold=sim_threshold,
        max_retries=max_retries,
        verbose=verbose
    )

    # Return the updated DataFrame and the confirmed DataFrame
    return df, confirmed_df

# Columns to check for LCCNs
cols_to_check = [
    'PoB Source',
    'Place of Death Source',
    'Place of Residence Source',
    'Occupation Source',
    'DoB Source',
    'DoD (P570) Source',
    'P26+P2562 Source'
]

# Apply and confirm LCCNs for each column
for col in cols_to_check:
    df_persons, confirmed_lccns = apply_and_confirm_lccn(
        df_persons,
        col,
        get_lccn_for_title,
        confirm_lccn_matches,
        delay=1.5,  # Delay between requests
        sim_threshold=95,
        max_retries=5,
        verbose=True
    )
    time.sleep(5)  # Add a delay of 5 seconds between processing each column
# Display the updated DataFrame
df_persons.head()

In [ ]:
df_persons.info()

# Convert df_persons columns to template

#Deduplicate df_persons

In [ ]:
#fuzzy matching tool
%pip install thefuzz
from thefuzz import fuzz, process
#example--produces similarity score
fuzz.ratio('contact zone-encounter', 'contact zones-encounter')

#Note: build auto-suggestions for csvs
#see process.extract and process.extractOne methods of the fuzz
#https://stackoverflow.com/questions/10383044/fuzzy-string-comparison


In [ ]:
# Create a list of all names in the DataFrame
names = df_persons['name'].dropna().tolist()

# Define a function to find potential duplicates
def find_duplicates(name):
    if not isinstance(name, str):
        return np.nan  # Return NaN if the name is not a string
    matches = process.extract(name, names, limit=None, scorer=fuzz.token_sort_ratio)  # Use token_sort_ratio
    potential_duplicates = [
        match[0] for match in matches if match[1] > 95
    ]  # Filter matches with token_sort_ratio > 95

    # If no matches with token_sort_ratio, try token_set_ratio
    if not potential_duplicates:
        matches = process.extract(name, names, limit=None, scorer=fuzz.token_set_ratio)  # Use token_set_ratio
        potential_duplicates = [
            match[0] for match in matches if match[1] > 95
        ]  # Filter matches with token_set_ratio > 95

    return potential_duplicates if potential_duplicates else np.nan

# Apply the function to create the 'Duplicate(s)?' column
df_persons['Duplicate(s)?'] = df_persons['name'].apply(find_duplicates)

In [ ]:
df_persons['Duplicate(s)?'].info()
df_persons['name'].info()

In [ ]:
df_persons[df_persons['Duplicate(s)?'].notna() & (df_persons['Duplicate(s)?'] != '')][['name', 'Duplicate(s)?']]

NOTE: IF THERE'S A DUPE, IT IS DROPPED!!!

In [ ]:
# cut all rows where there is a dupe? what's the process here?
df_persons = df_persons.drop(df_persons.dropna(subset=['Duplicate(s)?']).index)
#note what else is in here -- keep only rows where we have data in the columns needed for our LOD schema (need to get the schema)
df_persons.info()

In [ ]:
#DONT DO FOR NOW
#Drop all rows with wikidata info already
#df_persons = df_persons.drop(df_persons.dropna(subset=['LOD - WikiData']).index)

In [ ]:
df_persons.info()

In [ ]:
#Now that everything is deduplicated, make extra rows for records where columns that may
#contain more than one value have 2 or more values. The result will be for the original row to remain, but with the second, third, etc. values
#in particular columns missing. New rows will just have the name and the appropriate values.
#For example, if a record has two marriage dates, then once fixed it will have the first marriage date and source info in the original row,
#and a new row with only the name and the second marriage date and source info.
#This is to make working with OpenRefine easier
def create_new_rows(df):
    new_rows = []
    for index, row in df.iterrows():
        # Check for semicolons in 'Marriage Date' and 'Place of Residence'
        marriage_dates = str(row['Marriage Date']).split(';')
        places_of_residence = str(row['Place of Residence']).split(';')

        # Process 'Marriage Date' entries
        if len(marriage_dates) > 1:
            marriage_date_sources = str(row['P26+P2562 Source']).split(';')
            marriage_date_source_urls = str(row['P26+P2562 Source URL']).split(';')

            # Keep only the first marriage date in the original row
            df.loc[index, 'Marriage Date'] = marriage_dates[0].strip()
            df.loc[index, 'P26+P2562 Source'] = marriage_date_sources[0].strip() if marriage_date_sources else ''
            df.loc[index, 'P26+P2562 Source URL'] = marriage_date_source_urls[0].strip() if marriage_date_source_urls else ''

            # Create new rows for the remaining marriage dates
            for i in range(1, len(marriage_dates)):
                new_row = pd.Series({'name': row['name']})  # Start with just the name
                new_row['Marriage Date'] = marriage_dates[i].strip()
                new_row['P26+P2562 Source'] = marriage_date_sources[i].strip() if i < len(marriage_date_sources) else ''
                new_row['P26+P2562 Source URL'] = marriage_date_source_urls[i].strip() if i < len(marriage_date_source_urls) else ''
                new_rows.append(new_row)

        # Process 'Place of Residence' entries (similar logic to Marriage Date)
        if len(places_of_residence) > 1:
            residence_sources = str(row['Place of Residence Source']).split(';')
            residence_source_urls = str(row['Place of Residence Source URL']).split(';')

            df.loc[index, 'Place of Residence'] = places_of_residence[0].strip()
            df.loc[index, 'Place of Residence Source'] = residence_sources[0].strip() if residence_sources else ''
            df.loc[index, 'Place of Residence Source URL'] = residence_source_urls[0].strip() if residence_source_urls else ''

            for i in range(1, len(places_of_residence)):
                new_row = pd.Series({'name': row['name']})
                new_row['Place of Residence'] = places_of_residence[i].strip()
                new_row['Place of Residence Source'] = residence_sources[i].strip() if i < len(residence_sources) else ''
                new_row['Place of Residence Source URL'] = residence_source_urls[i].strip() if i < len(residence_source_urls) else ''
                new_rows.append(new_row)

    # Concatenate new rows with the original DataFrame
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    # Sort the DataFrame by 'name' column
    df = df.sort_values(by=['name'], ignore_index=True)
    return df

# Apply the function to create new rows
df_persons = create_new_rows(df_persons)

In [ ]:
# Export df_person to a CSV file with UTF-8-SIG encoding
df_persons_skeletal = df_persons[['name', 'AltLastName', 'AltMiddleName', 'AltFirstName', 'Maiden Name', 'Title', 'Birth Date', 'DoB Source', 'DoB Source URL', 'Death Date', 'DoD (P570) Source', 'DoD Source URL', 'Marriage Date', 'P26+P2562 Source', 'P26+P2562 Source URL', 'Source for Dates', 'Place of Birth (P19)', 'PoB Source', 'PoB Source URL', 'Place of Death', 'Place of Death Source', 'Place of Death Source URL', 'Place of Residence', 'Place of Residence Source', 'Place of Residence Source URL', 'Source for Places', 'Occupation', 'Occupation Source', 'Occupation Source URL', 'Gender', 'LOD - WikiData']]
df_persons_skeletal.to_csv('df_persons_skeletal.csv', index=False, encoding='utf-8-sig')

# Result is a df_persons_skeletal.csv with skeletal data

In [ ]:
df_persons_skeletal.info()

RELATIONSHIPS

In [ ]:
#get authority names for reduced list in df_persons
reduced_names = df_persons_skeletal['name'].dropna().tolist()
print(reduced_names)

In [ ]:
df_relationships.keys()

In [ ]:
#replace all na entries with a comma? for person fields in df_relationships (avoiding nan issue)
df_relationships.fillna({'Person 1 Last': ',', 'Person 1 Suffix': ',', 'Person 1 Prefix': ',', 'Person 1 First': ',', 'Person 2 Last': ',', 'Person 2 Suffix': ',', 'Person 2 Prefix': ',', 'Person 2 First': ','}, inplace=True)


In [ ]:
#match authority names from df_persons to rows in relationships missing authority names
p1_cols = ['Person 1 Last', 'Person 1 Suffix', 'Person 1 Prefix', 'Person 1 First']
p2_cols = ['Person 2 Last', 'Person 2 Suffix', 'Person 2 Prefix', 'Person 2 First']

#build mock names based on existing information
df_relationships['p1_mock_name'] = df_relationships[p1_cols].stack().groupby(level=0).agg(" ".join)
df_relationships['p2_mock_name'] = df_relationships[p2_cols].stack().groupby(level=0).agg(" ".join)

#now, remove commas so match is better
df_relationships['p1_mock_name'] = df_relationships['p1_mock_name'].str.replace(',', '', regex=True).str.replace('\s+', ' ', regex=True)
df_relationships['p2_mock_name'] = df_relationships['p2_mock_name'].str.replace(',', '', regex=True).str.replace('\s+', ' ', regex=True)
df_relationships['p1_mock_name'].tail(10)

#also, remove extra spaces and trim
df_relationships['p1_mock_name'] = df_relationships['p1_mock_name'].str.replace('\s+', ' ', regex=True).str.strip()
df_relationships['p2_mock_name'] = df_relationships['p2_mock_name'].str.replace('\s+', ' ', regex=True).str.strip()

In [ ]:
filtered_df_relationships = df_relationships[df_relationships['p1_std_name']==None]
df_relationships[['Person 1 Last', 'Person 1 First', 'p1_mock_name']].tail(50)

In [ ]:
# Function to find the best match from reduced_names
def find_best_match(mock_name, std_name):
    if isinstance(std_name, str):  # If std_name already exists, don't replace it
        return std_name
    if isinstance(mock_name, str):  # If mock_name is valid, find a match
        mock_name_cleaned = ' '.join(mock_name.split())  # Normalize whitespace
        match = process.extractOne(mock_name_cleaned, reduced_names)
        if match and match[1] >= 90:  # Match score >= 90
            return match[0]
    return " "  # Default value if no match is found

# Update p1_std_name column
df_relationships['p1_std_name'] = df_relationships.apply(
    lambda row: find_best_match(row['p1_mock_name'], row['p1_std_name']), axis=1
)

# Update p2_std_name column
df_relationships['p2_std_name'] = df_relationships.apply(
    lambda row: find_best_match(row['p2_mock_name'], row['p2_std_name']), axis=1
)

In [ ]:
df_relationships[['p1_std_name', 'p2_std_name', 'P1 Relation to P2']].head(60)

what to do if there aren't standard names for p1 and p2?
don't worry about it? what you're doing here is getting all the people data as columns in relationship tab; you are propagating anyways
how to approach the bubbling? build dict with keys as all mother-daughter, mother-son, father-daughter, father-son, brother-brother, brother-sister, sister-brother, sister-sister

In [ ]:
#first dup all rows with terms reversed
#then determine all relationships a person has as an array of tuples dict: key: person_name, value: array of tuples (person, relationship)
#then go through each person key, get array of tuples; go through array: for each tuple, search for a third party connection with a case statement

In [ ]:
#OLD NOT IN USE -- HAS USEFUL NESTED CASE/MATCH IN CASE CURRENT SCRIPT IS FAULTY
def propagate_relationships_dict(p_dict):
  p2_rels = []
  p2_people = []
  p2_other = []
  p1p3_rel=None
  p1=''
  p2=''
  p3=''

  # test: script should add ('Francis', 'father') to Bob (and eventually ('John', 'sister-in-law') to Mary)
  #has to be run twice I think
  p_dict_values = p_dict.values()
  print(p_dict_values)
  #add people who are not in the keys but in the values with just 'self' as a relation
  p_dict_new = [x[0] for x in p_dict_values if p_dict.get(x[0]) is None]
  for new_p in p_dict_new:
    p_dict[new_p] = [(new_p, 'self')]
  #print(p_dict)
  #loop through twice to sweep in and process added relationships
  for _ in range(2):
    for key, value in p_dict.items():
      p1=key
      for p2_tuple in value:
        p2=p2_tuple[0]
        p1p2_rel=p2_tuple[1]
        if (p_dict.get(p2) is not None) and (p1 != p2):
          p2p3_rels = [x[1] for i, x in enumerate(p_dict.get(p2))]
          p2p3_people =  [x[0] for i, x in enumerate(p_dict.get(p2))]
          for i, p2p3_rel in enumerate(p2p3_rels):
              #print(p1p2_rel, p2, p2p3_rel, p2p3_people[i])
              match p1p2_rel:
                case 'mother':
                  match p2p3_rel:
                    case 'mother'|'father':
                      p1p3_rel = 'grandmother'
                    case 'sister'|'brother':
                      p1p3_rel = 'mother'
                    case 'daughter'|'son':
                      p1p3_rel = 'wife'
                case 'father':
                  match p2p3_rel:
                    case 'mother'|'father':
                      p1p3_rel = 'grandfather'
                    case 'sister'|'brother':
                      p1p3_rel = 'father'
                    case 'daughter'|'son':
                      p1p3_rel = 'husband'
                case 'husband':
                  match p2p3_rel:
                    case 'mother':
                      p1p3_rel = 'father'
                    case 'sister'|'brother':
                      p1p3_rel = 'brother-in-law'
                    case 'daughter':
                      p1p3_rel = 'son-in-law'
                case 'wife':
                  match p2p3_rel:
                    case 'father':
                      p1p3_rel = 'mother'
                    case 'sister'|'brother':
                      p1p3_rel = 'sister-in-law'
                    case 'son':
                      p1p3_rel = 'daughter-in-law'
                case 'sister':
                  match p2p3_rel:
                    case 'brother'|'sister':
                      p1p3_rel='sister'
                    case 'father'|'mother':
                      p1p3_rel='aunt'
                    case 'son'|'daughter':
                      p1p3_rel='daughter'
                case 'brother':
                  match p2p3_rel:
                    case 'brother'|'sister':
                      p1p3_rel='brother'
                    case 'father'|'mother':
                      p1p3_rel='uncle'
                    case 'son'|'daughter':
                      p1p3_rel='son'
                case 'daughter':
                  match p2p3_rel:
                    case 'son'|'daughter':
                      p1p3_rel='granddaughter'
                    case 'sister'|'brother':
                      p1p3_rel='niece'
                    case 'mother'|'father':
                      p1p3_rel='sister'
                case 'son':
                  match p2p3_rel:
                    case 'son'|'daughter':
                      p1p3_rel='grandson'
                    case 'sister'|'brother':
                      p1p3_rel='nephew'
                    case 'mother'|'father':
                      p1p3_rel='brother'
              if p1p3_rel is not None:
                p_dict[p1].append((p2p3_people[i], p1p3_rel))
              p1p3_rel = None
          #do something
  for key, value in p_dict.items():
    p_dict[key]=list(set(value))
  #print(p_dict)
        #case 'wife':
        # print('here')
          #do something
        #case 'brother':
        #case 'sister':





In [ ]:
p_dict = {'Bob': [('Mary', 'husband'), ('John', 'brother')], 'Mary': [('Bob', 'wife'), ('Francis', 'mother')], 'Francis': [('George', 'sister'), ('Albert', 'mother')]}

In [ ]:
propagate_relationships_dict(p_dict)

In [ ]:
def propagate_relationships(df_relationships):
    """
    Propagates relationships in the DataFrame by:
    1. Adding missing reciprocal relationships.
    2. Inferring new relationships based on existing ones.
    """
    # Create a list to store new rows
    new_rows = []

    # Step 1: Add missing reciprocal relationships
    for _, row in df_relationships.iterrows():
        p1, p2, p1_to_p2, p2_to_p1 = row['p1_std_name'], row['p2_std_name'], row['P1 Relation to P2'], row['P2 Relation to P1']

        # Add reciprocal relationship for P1 -> P2
        if not ((df_relationships['p1_std_name'] == p2) &
                (df_relationships['p2_std_name'] == p1) &
                (df_relationships['P1 Relation to P2'] == p2_to_p1)).any():
            new_rows.append({'p1_std_name': p2, 'p2_std_name': p1, 'P1 Relation to P2': p2_to_p1, 'P2 Relation to P1': p1_to_p2})

    # Step 2: Infer relationships involving a third person
    for _, row1 in df_relationships.iterrows():
        p1, p2, p1_to_p2 = row1['p1_std_name'], row1['p2_std_name'], row1['P1 Relation to P2']
        for _, row2 in df_relationships.iterrows():
            if row1['p2_std_name'] == row2['p1_std_name']:  # Check if P2 in row1 is P1 in row2
                p3, p2_to_p3 = row2['p2_std_name'], row2['P1 Relation to P2']

                # Infer relationship between P1 and P3
                inferred_relation = None
                reverse_relation = None
                match p1_to_p2:
                    case 'mother':
                        match p2_to_p3:
                            case 'mother' | 'father':
                                inferred_relation, reverse_relation = 'grandmother', 'granddaughter'
                            case 'son' | 'daughter':
                                inferred_relation, reverse_relation = 'mother', 'child'
                    case 'father':
                        match p2_to_p3:
                            case 'mother' | 'father':
                                inferred_relation, reverse_relation = 'grandfather', 'grandson'
                            case 'son' | 'daughter':
                                inferred_relation, reverse_relation = 'father', 'child'
                    case 'husband':
                        match p2_to_p3:
                            case 'mother':
                                inferred_relation, reverse_relation = 'father', 'son'
                            case 'daughter':
                                inferred_relation, reverse_relation = 'son-in-law', 'father-in-law'
                    case 'wife':
                        match p2_to_p3:
                            case 'father':
                                inferred_relation, reverse_relation = 'mother', 'daughter'
                            case 'son':
                                inferred_relation, reverse_relation = 'daughter-in-law', 'mother-in-law'
                    case 'brother':
                        match p2_to_p3:
                            case 'brother' | 'sister':
                                inferred_relation, reverse_relation = 'brother', 'brother'
                            case 'son' | 'daughter':
                                inferred_relation, reverse_relation = 'uncle', 'nephew'
                    case 'sister':
                        match p2_to_p3:
                            case 'brother' | 'sister':
                                inferred_relation, reverse_relation = 'sister', 'sister'
                            case 'son' | 'daughter':
                                inferred_relation, reverse_relation = 'aunt', 'niece'

                # Add inferred relationship if applicable
                if inferred_relation:
                    if not ((df_relationships['p1_std_name'] == p1) &
                            (df_relationships['p2_std_name'] == p3) &
                            (df_relationships['P1 Relation to P2'] == inferred_relation)).any():
                        new_rows.append({'p1_std_name': p1, 'p2_std_name': p3, 'P1 Relation to P2': inferred_relation, 'P2 Relation to P1': reverse_relation})

                    # Add the reverse relationship
                    if not ((df_relationships['p1_std_name'] == p3) &
                            (df_relationships['p2_std_name'] == p1) &
                            (df_relationships['P1 Relation to P2'] == reverse_relation)).any():
                        new_rows.append({'p1_std_name': p3, 'p2_std_name': p1, 'P1 Relation to P2': reverse_relation, 'P2 Relation to P1': inferred_relation})

    # Step 3: Add new rows to the DataFrame
    if new_rows:
        df_relationships = pd.concat([df_relationships, pd.DataFrame(new_rows)], ignore_index=True)
    print(new_rows)
    return df_relationships

In [ ]:
propagate_relationships(df_relationships)

In [ ]:
# Remove rows where p1_std_name or p2_std_name do not contain any alphanumeric characters
df_relationships = df_relationships[
    df_relationships['p1_std_name'].str.contains(r'[A-Za-z0-9]', na=False) &
    df_relationships['p2_std_name'].str.contains(r'[A-Za-z0-9]', na=False)
]

In [ ]:
df_relationships.tail()

In [ ]:
# Export relationships as triples
# Create a new DataFrame with the desired columns
df_triples = df_relationships[['p1_std_name', 'P1 Relation to P2', 'p2_std_name']].rename(
    columns={'p1_std_name': 'Person1', 'P1 Relation to P2': 'Relationship', 'p2_std_name': 'Person2'}
)

# Export the DataFrame to a spreadsheet with UTF-8-SIG encoding
df_triples.to_csv('df_relationships_triples.csv', index=False, encoding='utf-8-sig')